# Forward-backward splitting for time-varying graphical lasso
This notebook shows how to minimise the time-varying graphical lasso with element-wise penalty norms across time-points.

First of all, as always, let's create a bunch of data.
For this task, we generate eah variable to change according to a certain behaviour which can be described as evolution via tigonometric functions, such as `sin` and `cos`.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from scipy.spatial.distance import squareform
from regain import datasets, utils

from sklearn.datasets import load_iris
from sklearn.svm import SVC 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, ShuffleSplit

from skopt.searchcv import BayesSearchCV
from skopt.space import Real, Categorical, Integer

In [ ]:
# np.random.seed(7)

# fs = 10e3
# N = 100
# amp = 2*np.sqrt(2)
# freq = 1.0
# noise_power = 0.001 * fs / 2
# time = np.arange(N) / fs
# z = amp*np.sin(2*np.pi*freq*time)
# z += np.random.normal(scale=np.sqrt(noise_power), size=time.shape)
# plt.plot(z);

In [ ]:
# T = 4

# x = np.tile(np.linspace(0, T-1, T), (n_interactions, 1))
# zz = amp * signal.square(2 * np.pi * freq * x + phase, duty=.5)
# plt.plot(x.T, zz.T);

Generate the data starting from the inverse covariance matrices.

In [ ]:
from regain.datasets import *

In [ ]:
def make_sin(n_dim_obs, n_dim_lat, T, shape='smooth', closeness=1,
             normalize=False):
    upper_idx = np.triu_indices(n_dim_obs, 1)
    n_interactions = len(upper_idx[0])
    x = np.tile(np.linspace(0, (T-1.) / closeness, T), (n_interactions, 1))
    phase = np.random.rand(n_interactions, 1)
    freq = np.random.rand(n_interactions, 1) - .250
    A = (np.random.rand(n_interactions, 1) + 2) / 2.

    if shape == 'smooth':
        y = A * np.sin(2. * np.pi * freq * x + phase)
    else:
        y = A * signal.square(2 * np.pi * freq * x + phase, duty=.5)

    # threshold
    y = np.maximum(y, 0)

    Y = np.array([squareform(y[:, j]) + np.diag(np.sum(squareform(y[:, j]), axis=1))
                  for j in range(y.shape[1])])

    if normalize:
        map(normalize_matrix, Y)  # in place
    assert positive_definite(Y)

    return Y, Y, np.zeros_like(Y)

data = {}
np.random.seed(7)
data['square'] = datasets.make_dataset(n_samples=n_samples, n_dim_obs=n_dim_obs, n_dim_lat=0, T=T,
                             time_on_axis='last',
                             mode=make_sin, shape='square', closeness=2.4, normalize=1)


plt.step(np.array([squareform(y, checks=None) for y in data['square'].thetas]), '-|');

In [ ]:
np.random.seed(7)

# square
n_samples = 100
n_dim_obs = 20
T = 10

data = {}
data['square'] = datasets.make_dataset(n_samples=n_samples, n_dim_obs=n_dim_obs, n_dim_lat=0, T=T,
                             time_on_axis='last',
                             mode='sin', shape='square', closeness=2.4, normalize=1)

# smooth
n_samples = 100
n_dim_obs = 20
T = 10

data['smooth'] = datasets.make_dataset(n_samples=n_samples, n_dim_obs=n_dim_obs, n_dim_lat=0, T=T,
                             time_on_axis='last',
                             mode='sin', shape='smooth', closeness=2.4, normalize=1)

plt.step(np.array([squareform(y, checks=None) for y in data['square'].thetas]), '-|');
# plt.savefig("/home/fede/Dropbox/Latent variables networks/forward backward time varying graphical lasso/smooth_signal.pdf")

### Let's run 

In [ ]:
# X = data.data
# X_tr, X_ts = train_test_split(X)

In [ ]:
import pandas as pd
from functools import partial
from regain import update_rules; reload(update_rules);
from regain.forward_backward import time_graph_lasso_; reload(time_graph_lasso_)

from sklearn.covariance import GraphLasso, GraphLassoCV
import sys; sys.path.append("/home/fede/src/TVGL")
import inferGraphL1; reload(inferGraphL1)
import inferGraphL2; reload(inferGraphL2)
import TVGL; reload(TVGL)

# use:
# beta = 2.1, norm = 1
# beta = 5.05, norm = 2
# prepare dataframe for results

methods = ['TGL-FB ($\ell_{21}$)', 'TGL-FB ($\ell_1$)', 'GL', 'TVGL ($\ell_{21}$)', 'TVGL ($\ell_1$)']

scores = sorted(['iter', 'accuracy',  'average_precision',  'balanced_accuracy','f1','false_omission_rate','fdr',
                 'fn',  'fp',  'precision',  'prevalence',  'recall',  'specificity',  'tn',  'tp'])
evolution = sorted(['square', 'smooth'])

rows = methods
cols = pd.MultiIndex.from_product([evolution, scores], names=('evolution', 'score'))
# rows = pd.MultiIndex.from_product([methods, n_times], names=('method','time'))

dff = pd.DataFrame(columns=cols, index=rows)
idx = pd.IndexSlice

In [ ]:
def run_performance(beta=1, alpha=1, evolution='square'):
    X = data[evolution].data
    
    error_function = partial(utils.structure_error, data[evolution].thetas,
                             no_diagonal=0, thresholding=1, eps=1e-4)

    tglfb = time_graph_lasso_.TimeGraphLassoForwardBackward(
        verbose=0, gamma=1, alpha=alpha, beta=beta,
        delta=1e-8, choose='lamda', lamda_criterion='c',
        lamda=1, tol=1e-5, eps=0.9,
        time_norm=2, max_iter=300, time_on_axis='last').fit(X)
#     print(tglfb.alpha_max(X))
#     return
    res = error_function(tglfb.precision_)
    res['iter'] = tglfb.n_iter_
    dff.loc['TGL-FB ($\ell_{21}$)', idx[evolution, :]] = [res[x] for x in scores]
    
    tglfb = tglfb.set_params(time_norm=1).fit(X)
    res = error_function(tglfb.precision_)
    res['iter'] = tglfb.n_iter_
    dff.loc['TGL-FB ($\ell_1$)', idx[evolution, :]] = [res[x] for x in scores]

    gl = GraphLasso(alpha=0.05)
#     gl = GraphLassoCV(alphas=list(np.logspace(-1, 0, 100)))
    precision_gl = np.array([gl.fit(x).precision_.copy() for x in X.transpose(2,0,1)])
    res = error_function(precision_gl)
    res['iter'] = gl.n_iter_
    dff.loc['GL', idx[evolution, :]] = [res[x] for x in scores]
    
    thetaSet, empCovSet, status, gvx = TVGL.TVGL(
        np.vstack(X.transpose(2,0,1)), X.shape[0],
        lamb=gl.alpha, beta=tglfb.beta, indexOfPenalty=2, verbose=False)

    res = error_function(np.array(thetaSet))
    res['iter'] = gvx.n_iter_
    dff.loc['TVGL ($\ell_{21}$)', idx[evolution, :]] = [res[x] for x in scores]
    
    thetaSet, empCovSet, status, gvx = TVGL.TVGL(
        np.vstack(X.transpose(2,0,1)), X.shape[0],
        lamb=gl.alpha, beta=tglfb.beta, indexOfPenalty=1, verbose=False)
    res = error_function(np.array(thetaSet))
    res['iter'] = gvx.n_iter_
    dff.loc['TVGL ($\ell_1$)', idx[evolution, :]] = [res[x] for x in scores]

In [ ]:
run_performance(beta=1.09, alpha=0.55, evolution='square')
# run_performance(beta=5, alpha=0.53, evolution='smooth')

In [ ]:
ss = ['accuracy', 'average_precision', 'balanced_accuracy', 'f1', 'iter']
# dff['smooth'].sort_values("f1", ascending=False)#[ss]
dff['square'].sort_values("f1", ascending=False)

In [ ]:
# dff.to_pickle("results_25_04_18.pkl")

In [ ]:
evolution = 'square'; alpha = 0.55; beta = 1.09
# evolution = 'smooth'; alpha = 0.53; beta = 5
from regain import prox; reload(prox)
from regain import norm; reload(norm)
reload(time_graph_lasso_)
X = data[evolution].data


tglfb_square = time_graph_lasso_.TimeGraphLassoForwardBackward(
    verbose=2, gamma=1, alpha=alpha, beta=beta,
    delta=1e-8, choose='lamda', lamda_criterion='c',
    lamda=1, tol=1e-5, eps=0.9, return_history=True,
    time_norm=1, max_iter=300, time_on_axis='last').fit(X)

tglfb_smooth = time_graph_lasso_.TimeGraphLassoForwardBackward(
    verbose=2, gamma=1, alpha=alpha, beta=beta,
    delta=1e-8, choose='lamda', lamda_criterion='c',
    lamda=1, tol=1e-5, eps=0.9, return_history=True,
    time_norm=2, max_iter=300, time_on_axis='last').fit(X)

# thetaSet, empCovSet, status, gvx_square = TVGL.TVGL(
#     np.vstack(X.transpose(2,0,1)), X.shape[0],
#     lamb=alpha, beta=beta, indexOfPenalty=2, verbose=False, epsAbs=1e-5)
# thetaSet, empCovSet, status, gvx_smooth = TVGL.TVGL(
#     np.vstack(X.transpose(2,0,1)), X.shape[0],
#     lamb=alpha, beta=beta, indexOfPenalty=1, verbose=False, epsAbs=1e-5)

plt.title("Evolution: %s" % evolution)
plt.semilogy([x.rnorm for x in tglfb_smooth.history_], label='TVGL $\ell_{21}$')
plt.semilogy([x.rnorm for x in tglfb_square.history_], label='TGL-FB $\ell_1$')
plt.semilogy([x for x in gvx_square.history_], label='TVGL $\ell_1$')
plt.semilogy([x for x in gvx_smooth.history_], label='TVGL $\ell_{21}$')
plt.legend()

In [ ]:
plt.title("Evolution: %s" % evolution)
plt.plot([x.rnorm for x in tglfb_smooth.history_], label='TVGL $\ell_{21}$')
plt.plot([x.rnorm for x in tglfb_square.history_], label='TGL-FB $\ell_1$')
plt.plot([x for x in gvx_square.history_], label='TVGL $\ell_1$')
plt.plot([x for x in gvx_smooth.history_], label='TVGL $\ell_{21}$')
plt.legend()
# plt.xlim([0,5])

In [ ]:
error_function = partial(utils.structure_error, data[evolution].thetas,
                             no_diagonal=0, thresholding=1, eps=1e-4)

res = error_function(tglfb.precision_)
res

In [ ]:
tglfb.n_iter_

In [ ]:
np.abs(tglfb.precision_).min()

### BayesOptimisation
Since we have lots of hyper-parameters, we rely on a Bayesian optimisation procedure in order to select the best hyper-parameters, treating the scoring function of our algorithm as a black-box for the gaussian process underlying the Bayesian optimisation.

Such procedure is performed via the `scikit-optimize` package.

In [ ]:
from regain import utils; reload(utils)
from regain import prox; reload(prox);
from regain.forward_backward import time_graph_lasso_; reload(time_graph_lasso_)

from skopt import searchcv; reload(searchcv)

X = data['square'].data

domain = {'alpha': Real(1e-1, 3, prior='uniform'),
          'beta': Real(1e-1, 1e1, prior='uniform'),
#           'time_norm': Categorical([1, 2])
#           'eps': Categorical([0.5, 0.7, 0.8, 0.9])
         }

mdl = time_graph_lasso_.TimeGraphLassoForwardBackward(
    verbose=0, tol=1e-5, max_iter=200, gamma=1, beta=2.1, time_norm=1,
    time_on_axis='last', alpha=.77,
    delta=1e-8, choose='lamda', lamda_criterion='c', eps=0.9)
    
cv = ShuffleSplit(5, test_size=0.2)
    
bscv = searchcv.BayesSearchCV(
    mdl, domain, n_iter=100, cv=cv, verbose=0, n_jobs=1, iid=True, n_points=3,
    error_score=-3e5)

def on_step(optim_result):
    score = bscv.best_score_
#     print("best score: %s" % score)

# bscv.fit(X, callback=on_step)

In [ ]:
bscv_smooth.best_params_

In [ ]:
# searchCV on smooth data set
domain = {'alpha': Real(0.1, 1, prior='uniform'),
          'beta': Real(0.5, 5, prior='uniform'),
#           'time_norm': Categorical([1, 2])
#           'eps': Categorical([0.5, 0.7, 0.8, 0.9])
         }
bscv_smooth = searchcv.BayesSearchCV(
    mdl, domain, n_iter=300, cv=cv, verbose=0, n_jobs=1, iid=True, n_points=3,
    error_score=-3e5)

def on_step(optim_result):
    score = bscv_smooth.best_score_
#     print("best score: %s" % score)

ltgl.fit(data_grid, callback=on_step)
# mdl.fit(data_grid)

In [ ]:
utils.structure_error(data.thetas, mdl.precision_, no_diagonal=0, thresholding=1, eps=1e-8)

In [ ]:
mdl = ltgl.best_estimator_

In [ ]:
mdl.score(data_grid)

In [ ]:
ltgl.cv_results_

In [ ]:
ltgl.best_params_

In [ ]:
bscv_smooth.fit(data['smooth'].data, callback=on_step)

### GridSearchCV
As for the hyper-parameters tuning, one may choose to fix a grid of parameters and select the best ones.
For this we can use `GridSearchCV`, from the `scikit-learn` library.

In [ ]:
param_grid=dict(alpha=np.logspace(-2,0,3), beta=np.logspace(-2,0,3),
                time_norm=[1, 2])

mdl = time_graph_lasso_.TimeGraphLassoForwardBackward(
    verbose=0, time_on_axis='last')
    
cv = ShuffleSplit(2, test_size=0.2)
ltgl = GridSearchCV(mdl, param_grid, cv=cv, verbose=1)
ltgl.fit(data_grid)